<a href="https://colab.research.google.com/github/janaghoniem/Social-Media-Sentiment-Analysis/blob/main/ML_Arabic%26English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import vstack
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import pandas as pd
import numpy as np
import ast
import re
import pickle
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive

### Arabic Testing

In [3]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Science/Arabic Sentiment/Datasets/arabic_sentiment_reviews.csv')
with open("/content/drive/MyDrive/Colab Notebooks/Data Science/Arabic Sentiment/Features/training_data.pkl", 'rb') as f:
    training_data = pickle.load(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1) Random forest

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder, label_binarize
import tensorflow as tf

# Load features and labels
X = training_data['X_fasttext']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])
y_cat = tf.keras.utils.to_categorical(y)

# Split data into train, validation, test
X_temp, X_test, y_temp, y_test = train_test_split(X, y_cat, test_size=0.2, stratify=y_cat, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# Prepare labels for sklearn (not one-hot)
y_train_sklearn = np.argmax(y_train, axis=1)
y_val_sklearn = np.argmax(y_val, axis=1)
y_test_sklearn = np.argmax(y_test, axis=1)

# Train initial Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
rf_model.fit(X_train, y_train_sklearn)

# Evaluate initial model
print("Initial Random Forest Training Accuracy:", accuracy_score(y_train_sklearn, rf_model.predict(X_train)))
print("Initial Random Forest Validation Accuracy:", accuracy_score(y_val_sklearn, rf_model.predict(X_val)))
print("Initial Random Forest Testing Accuracy:", accuracy_score(y_test_sklearn, rf_model.predict(X_test)))

print("--------------------------------------------------")

# Combine train and validation for tuning
X_train_val = np.vstack([X_train, X_val])
y_train_val = np.concatenate([y_train_sklearn, y_val_sklearn])

# Hyperparameter tuning with GridSearchCV
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [20, 30, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_grid = GridSearchCV(RandomForestClassifier(random_state=42),
                       rf_params,
                       cv=3,
                       n_jobs=-1,
                       verbose=1)
rf_grid.fit(X_train_val, y_train_val)

# Best model after tuning
best_rf = rf_grid.best_estimator_

# Evaluate best model
print("Best Parameters:", rf_grid.best_params_)
print("Best Random Forest Training Accuracy:", accuracy_score(y_train_sklearn, best_rf.predict(X_train)))
print("Best Random Forest Validation Accuracy:", accuracy_score(y_val_sklearn, best_rf.predict(X_val)))
print("Best Random Forest Testing Accuracy:", accuracy_score(y_test_sklearn, best_rf.predict(X_test)))

print("--------------------------------------------------")

# Evaluate best model with additional metrics on the test set
y_test_pred = best_rf.predict(X_test)
y_test_proba = best_rf.predict_proba(X_test)

# Accuracy
print("Test Accuracy:", accuracy_score(y_test_sklearn, y_test_pred))

# Precision, Recall, F1-score (macro because it's multiclass)
print("Test Precision (Macro):", precision_score(y_test_sklearn, y_test_pred, average='macro'))
print("Test Recall (Macro):", recall_score(y_test_sklearn, y_test_pred, average='macro'))
print("Test F1-Score (Macro):", f1_score(y_test_sklearn, y_test_pred, average='macro'))

# Classification report
print("\nClassification Report:\n", classification_report(y_test_sklearn, y_test_pred, target_names=label_encoder.classes_))

# AUC-ROC (requires one-hot encoded y_test)
y_test_bin = label_binarize(y_test_sklearn, classes=np.arange(len(label_encoder.classes_)))

# Handle multiclass AUC
try:
    auc_roc = roc_auc_score(y_test_bin, y_test_proba, multi_class='ovr', average='macro')
    print("Test AUC-ROC (Macro OVR):", auc_roc)
except ValueError as e:
    print("AUC-ROC Error:", e)


Initial Random Forest Training Accuracy: 0.9956228956228956
Initial Random Forest Validation Accuracy: 0.7590530303030303
Initial Random Forest Testing Accuracy: 0.7500151515151515
--------------------------------------------------
Fitting 3 folds for each of 24 candidates, totalling 72 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



2) SVM

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Load features and labels
X = training_data['X_fasttext']  # Use your dense FastText embeddings
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])
y_cat = tf.keras.utils.to_categorical(y)

# Split into train, validation, test (stratifying on labels)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# ----------------- Baseline SVM -----------------
print("\nTraining Baseline SVM...")
svm_baseline = LinearSVC(max_iter=10000, random_state=42, dual=False)
svm_baseline.fit(X_train, y_train)

# Predictions
train_preds = svm_baseline.predict(X_train)
val_preds = svm_baseline.predict(X_val)
test_preds = svm_baseline.predict(X_test)

print("\nBaseline SVM Results:")
print("Training Accuracy:", accuracy_score(y_train, train_preds))
print("Validation Accuracy:", accuracy_score(y_val, val_preds))
print("Testing Accuracy:", accuracy_score(y_test, test_preds))

# ----------------- SVM Hyperparameter Tuning -----------------
print("\nStarting SVM hyperparameter tuning...")
svm_params = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2'],
    'loss': ['squared_hinge'],
    'dual': [False]
}

svm_grid = GridSearchCV(
    LinearSVC(random_state=42, max_iter=10000),
    svm_params,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit ONLY on X_train (avoid stacking with val)
svm_grid.fit(X_train, y_train)
best_svm = svm_grid.best_estimator_

# Predictions after tuning
svm_train_preds = best_svm.predict(X_train)
svm_val_preds = best_svm.predict(X_val)
svm_test_preds = best_svm.predict(X_test)

print("\nTuned SVM Results:")
print("Best Parameters:", svm_grid.best_params_)
print("Training Accuracy:", accuracy_score(y_train, svm_train_preds))
print("Validation Accuracy:", accuracy_score(y_val, svm_val_preds))
print("Testing Accuracy:", accuracy_score(y_test, svm_test_preds))



Training Baseline SVM...

Baseline SVM Results:
Training Accuracy: 0.798956228956229
Validation Accuracy: 0.7990909090909091
Testing Accuracy: 0.7961818181818182

Starting SVM hyperparameter tuning...
Fitting 3 folds for each of 4 candidates, totalling 12 fits

Tuned SVM Results:
Best Parameters: {'C': 10, 'dual': False, 'loss': 'squared_hinge', 'penalty': 'l2'}
Training Accuracy: 0.7989814814814815
Validation Accuracy: 0.7992803030303031
Testing Accuracy: 0.7964545454545454


In [11]:
from sklearn.metrics import classification_report, roc_auc_score

# ---- Classification Report ----
print("\nClassification Report (Test Data):")
print(classification_report(y_test, test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (using decision_function) ----
try:
    test_decision_scores = svm_baseline.decision_function(X_test)
    auc = roc_auc_score(y_test, test_decision_scores, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR):", auc)
except Exception as e:
    print("AUC-ROC could not be calculated:", e)

from sklearn.metrics import classification_report, roc_auc_score

# ---- Classification Report (Tuned SVM) ----
print("\nClassification Report (Tuned SVM - Test Data):")
print(classification_report(y_test, svm_test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (using decision_function for Tuned SVM) ----
try:
    test_decision_scores_tuned = best_svm.decision_function(X_test)
    auc_tuned = roc_auc_score(y_test, test_decision_scores_tuned, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Tuned SVM:", auc_tuned)
except Exception as e:
    print("AUC-ROC (Tuned SVM) could not be calculated:", e)




Classification Report (Test Data):
              precision    recall  f1-score   support

           0       0.61      0.77      0.68     32629
           1       0.70      0.52      0.60     33371

    accuracy                           0.64     66000
   macro avg       0.65      0.64      0.64     66000
weighted avg       0.65      0.64      0.64     66000

AUC-ROC (macro average OvR): 0.8760152659478608

Classification Report (Tuned SVM - Test Data):
              precision    recall  f1-score   support

           0       0.79      0.81      0.80     32629
           1       0.81      0.79      0.80     33371

    accuracy                           0.80     66000
   macro avg       0.80      0.80      0.80     66000
weighted avg       0.80      0.80      0.80     66000

AUC-ROC (macro average OvR) Tuned SVM: 0.8760306967319824


3) Logistic regression

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Load features and labels
X = training_data['X_fasttext']  # Use your dense FastText embeddings
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])
y_cat = tf.keras.utils.to_categorical(y)

# Split into train, validation, test (stratifying on labels)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# ----------------- Baseline Logistic Regression -----------------
print("\nTraining Baseline Logistic Regression...")
logreg_baseline = LogisticRegression(max_iter=10000, random_state=42, solver='lbfgs', multi_class='multinomial')
logreg_baseline.fit(X_train, y_train)

# Predictions
train_preds = logreg_baseline.predict(X_train)
val_preds = logreg_baseline.predict(X_val)
test_preds = logreg_baseline.predict(X_test)

print("\nBaseline Logistic Regression Results:")
print("Training Accuracy:", accuracy_score(y_train, train_preds))
print("Validation Accuracy:", accuracy_score(y_val, val_preds))
print("Testing Accuracy:", accuracy_score(y_test, test_preds))

# ----------------- Logistic Regression Hyperparameter Tuning -----------------
print("\nStarting Logistic Regression hyperparameter tuning...")
logreg_params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs'],  # Good for multiclass
    'multi_class': ['multinomial']
}

logreg_grid = GridSearchCV(
    LogisticRegression(max_iter=10000, random_state=42),
    logreg_params,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit ONLY on X_train
logreg_grid.fit(X_train, y_train)
best_logreg = logreg_grid.best_estimator_

# Predictions after tuning
logreg_train_preds = best_logreg.predict(X_train)
logreg_val_preds = best_logreg.predict(X_val)
logreg_test_preds = best_logreg.predict(X_test)

print("\nTuned Logistic Regression Results:")
print("Best Parameters:", logreg_grid.best_params_)
print("Training Accuracy:", accuracy_score(y_train, logreg_train_preds))
print("Validation Accuracy:", accuracy_score(y_val, logreg_val_preds))
print("Testing Accuracy:", accuracy_score(y_test, logreg_test_preds))



Training Baseline Logistic Regression...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(



Baseline Logistic Regression Results:
Training Accuracy: 0.796986531986532
Validation Accuracy: 0.7985227272727272
Testing Accuracy: 0.7941666666666667

Starting Logistic Regression hyperparameter tuning...
Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(



Tuned Logistic Regression Results:
Best Parameters: {'C': 100, 'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'lbfgs'}
Training Accuracy: 0.7981060606060606
Validation Accuracy: 0.798939393939394
Testing Accuracy: 0.7962575757575757


In [13]:
from sklearn.metrics import classification_report, roc_auc_score

# ---- Classification Report (Baseline Logistic Regression) ----
print("\nClassification Report (Baseline Logistic Regression - Test Data):")
print(classification_report(y_test, test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (using decision_function for Baseline Logistic Regression) ----
try:
    test_decision_scores_baseline = logreg_baseline.decision_function(X_test)
    auc_baseline = roc_auc_score(y_test, test_decision_scores_baseline, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Baseline Logistic Regression:", auc_baseline)
except Exception as e:
    print("AUC-ROC (Baseline Logistic Regression) could not be calculated:", e)


# ---- Classification Report (Tuned Logistic Regression) ----
print("\nClassification Report (Tuned Logistic Regression - Test Data):")
print(classification_report(y_test, logreg_test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (using decision_function for Tuned Logistic Regression) ----
try:
    test_decision_scores_tuned = best_logreg.decision_function(X_test)
    auc_tuned = roc_auc_score(y_test, test_decision_scores_tuned, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Tuned Logistic Regression:", auc_tuned)
except Exception as e:
    print("AUC-ROC (Tuned Logistic Regression) could not be calculated:", e)



Classification Report (Baseline Logistic Regression - Test Data):
              precision    recall  f1-score   support

           0       0.61      0.77      0.68     32629
           1       0.70      0.52      0.60     33371

    accuracy                           0.64     66000
   macro avg       0.65      0.64      0.64     66000
weighted avg       0.65      0.64      0.64     66000

AUC-ROC (macro average OvR) Baseline Logistic Regression: 0.8741980417747179

Classification Report (Tuned Logistic Regression - Test Data):
              precision    recall  f1-score   support

           0       0.79      0.80      0.80     32629
           1       0.81      0.79      0.80     33371

    accuracy                           0.80     66000
   macro avg       0.80      0.80      0.80     66000
weighted avg       0.80      0.80      0.80     66000

AUC-ROC (macro average OvR) Tuned Logistic Regression: 0.875517256263241


4) Naive Bayes

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Load features and labels
X = training_data['X_fasttext']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

# Split into train, validation, test (stratifying on labels)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# ----------------- Baseline Gaussian Naive Bayes -----------------
print("\nTraining Baseline Gaussian Naive Bayes...")
nb_baseline = GaussianNB()
nb_baseline.fit(X_train, y_train)

# Predictions
train_preds = nb_baseline.predict(X_train)
val_preds = nb_baseline.predict(X_val)
test_preds = nb_baseline.predict(X_test)

print("\nBaseline Gaussian Naive Bayes Results:")
print("Training Accuracy:", accuracy_score(y_train, train_preds))
print("Validation Accuracy:", accuracy_score(y_val, val_preds))
print("Testing Accuracy:", accuracy_score(y_test, test_preds))

# ----------------- Naive Bayes Hyperparameter Tuning -----------------
# For GaussianNB, the main hyperparameter is var_smoothing (default: 1e-9)
print("\nStarting Gaussian Naive Bayes hyperparameter tuning...")
nb_params = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

nb_grid = GridSearchCV(
    GaussianNB(),
    nb_params,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit ONLY on X_train
nb_grid.fit(X_train, y_train)
best_nb = nb_grid.best_estimator_

# Predictions after tuning
nb_train_preds = best_nb.predict(X_train)
nb_val_preds = best_nb.predict(X_val)
nb_test_preds = best_nb.predict(X_test)

print("\nTuned Gaussian Naive Bayes Results:")
print("Best Parameters:", nb_grid.best_params_)
print("Training Accuracy:", accuracy_score(y_train, nb_train_preds))
print("Validation Accuracy:", accuracy_score(y_val, nb_val_preds))
print("Testing Accuracy:", accuracy_score(y_test, nb_test_preds))



Training Baseline Gaussian Naive Bayes...

Baseline Gaussian Naive Bayes Results:
Training Accuracy: 0.6434175084175084
Validation Accuracy: 0.6449621212121213
Testing Accuracy: 0.6429090909090909

Starting Gaussian Naive Bayes hyperparameter tuning...
Fitting 3 folds for each of 5 candidates, totalling 15 fits

Tuned Gaussian Naive Bayes Results:
Best Parameters: {'var_smoothing': 1e-05}
Training Accuracy: 0.6433964646464646
Validation Accuracy: 0.645
Testing Accuracy: 0.6428939393939394


In [17]:
from sklearn.metrics import classification_report, roc_auc_score

# ---- Classification Report (Baseline Naive Bayes) ----
print("\nClassification Report (Baseline Gaussian Naive Bayes - Test Data):")
print(classification_report(y_test, test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (Baseline Naive Bayes) ----
try:
    test_prob_scores_baseline = nb_baseline.predict_proba(X_test)
    auc_baseline = roc_auc_score(y_test, test_prob_scores_baseline, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Baseline Naive Bayes:", auc_baseline)
except Exception as e:
    print("AUC-ROC (Baseline Naive Bayes) could not be calculated:", e)

# ---- Classification Report (Tuned Naive Bayes) ----
print("\nClassification Report (Tuned Gaussian Naive Bayes - Test Data):")
print(classification_report(y_test, nb_test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (Tuned Naive Bayes) ----
try:
    test_prob_scores_tuned = best_nb.predict_proba(X_test)
    auc_tuned = roc_auc_score(y_test, test_prob_scores_tuned, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Tuned Naive Bayes:", auc_tuned)
except Exception as e:
    print("AUC-ROC (Tuned Naive Bayes) could not be calculated:", e)




Classification Report (Baseline Gaussian Naive Bayes - Test Data):
              precision    recall  f1-score   support

           0       0.61      0.77      0.68     32629
           1       0.70      0.52      0.60     33371

    accuracy                           0.64     66000
   macro avg       0.65      0.64      0.64     66000
weighted avg       0.65      0.64      0.64     66000

AUC-ROC (Baseline Naive Bayes) could not be calculated: y should be a 1d array, got an array of shape (66000, 2) instead.

Classification Report (Tuned Gaussian Naive Bayes - Test Data):
              precision    recall  f1-score   support

           0       0.61      0.77      0.68     32629
           1       0.70      0.52      0.60     33371

    accuracy                           0.64     66000
   macro avg       0.65      0.64      0.64     66000
weighted avg       0.65      0.64      0.64     66000

AUC-ROC (Tuned Naive Bayes) could not be calculated: y should be a 1d array, got an array o

### English testing

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/English_cleaned.csv')
X = df['normalized_tweet']
y = df['target']

1) Random Forest

In [ ]:
# Split into train+validation and test sets first (80% train_val, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Then split train+validation into train and validation sets (80/20 split of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)  # 0.25 x 0.8 = 0.2, so final: 60% train, 20% val, 20% test

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Random Forest Classifier
rf_model = RandomForestClassifier(
    n_estimators=100,  # Number of trees
    random_state=42,
    n_jobs=-1  # Use all cores for training
)
rf_model.fit(X_train_tfidf, y_train)

# Predictions
y_train_pred = rf_model.predict(X_train_tfidf)
y_val_pred = rf_model.predict(X_val_tfidf)
y_test_pred = rf_model.predict(X_test_tfidf)

# Accuracy calculations
train_acc = accuracy_score(y_train, y_train_pred)
val_acc = accuracy_score(y_val, y_val_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# Output results
print(f"Random Forest Train Accuracy: {train_acc:.4f}")
print(f"Random Forest Validation Accuracy: {val_acc:.4f}")
print(f"Random Forest Test Accuracy: {test_acc:.4f}")

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Fit grid search to the training data
grid_search.fit(X_train_tfidf, y_train)

# Best parameters and model
print("Best Hyperparameters:", grid_search.best_params_)
best_rf_model = grid_search.best_estimator_

# Evaluate tuned model
y_train_pred_tuned = best_rf_model.predict(X_train_tfidf)
y_val_pred_tuned = best_rf_model.predict(X_val_tfidf)
y_test_pred_tuned = best_rf_model.predict(X_test_tfidf)

# Accuracy calculations for tuned model
train_acc_tuned = accuracy_score(y_train, y_train_pred_tuned)
val_acc_tuned = accuracy_score(y_val, y_val_pred_tuned)
test_acc_tuned = accuracy_score(y_test, y_test_pred_tuned)

# Output tuned model results
print(f"TUNED Random Forest Train Accuracy: {train_acc_tuned:.4f}")
print(f"TUNED Random Forest Validation Accuracy: {val_acc_tuned:.4f}")
print(f"TUNED Random Forest Test Accuracy: {test_acc_tuned:.4f}")

SVM Train Accuracy: 0.7924
SVM Validation Accuracy: 0.7255
SVM Test Accuracy: 0.7256
Fitting 3 folds for each of 16 candidates, totalling 48 fits


KeyboardInterrupt: 

2) SVM

In [ ]:
# Split into train+validation and test sets first (80% train_val, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Then split train+validation into train and validation sets (80/20 split of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)  # Final: 60% train, 20% val, 20% test

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Linear SVM Model
svm_model = LinearSVC(max_iter=10000, random_state=42)  # Increased max_iter for stability
svm_model.fit(X_train_tfidf, y_train)

# Predictions
y_train_pred = svm_model.predict(X_train_tfidf)
y_val_pred = svm_model.predict(X_val_tfidf)
y_test_pred = svm_model.predict(X_test_tfidf)

# Accuracy calculations
train_acc = accuracy_score(y_train, y_train_pred)
val_acc = accuracy_score(y_val, y_val_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# Output results
print(f"SVM Train Accuracy: {train_acc:.4f}")
print(f"SVM Validation Accuracy: {val_acc:.4f}")
print(f"SVM Test Accuracy: {test_acc:.4f}")

# Define parameter grid
svm_param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'loss': ['hinge', 'squared_hinge'],
    'max_iter': [20000, 30000]
}

# Set up GridSearchCV
svm_grid_search = GridSearchCV(
    LinearSVC(random_state=42),
    svm_param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Fit grid search to training data
svm_grid_search.fit(X_train_tfidf, y_train)

# Best parameters and model
print("Best SVM Hyperparameters:", svm_grid_search.best_params_)
best_svm_model = svm_grid_search.best_estimator_

# Predictions using the tuned model
y_train_pred_tuned = best_svm_model.predict(X_train_tfidf)
y_val_pred_tuned = best_svm_model.predict(X_val_tfidf)
y_test_pred_tuned = best_svm_model.predict(X_test_tfidf)

# Accuracy calculations for tuned model
train_acc_tuned = accuracy_score(y_train, y_train_pred_tuned)
val_acc_tuned = accuracy_score(y_val, y_val_pred_tuned)
test_acc_tuned = accuracy_score(y_test, y_test_pred_tuned)

# Output tuned model results
print(f"TUNED SVM Train Accuracy: {train_acc_tuned:.4f}")
print(f"TUNED SVM Validation Accuracy: {val_acc_tuned:.4f}")
print(f"TUNED SVM Test Accuracy: {test_acc_tuned:.4f}")

KeyboardInterrupt: 

3) Logistic Regression

In [ ]:
# Split into train+validation and test sets first (80% train_val, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Then split train+validation into train and validation sets (80/20 split of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)  # 0.25 x 0.8 = 0.2, so final: 60% train, 20% val, 20% test

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Logistic Regression Model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Predictions
y_train_pred = model.predict(X_train_tfidf)
y_val_pred = model.predict(X_val_tfidf)
y_test_pred = model.predict(X_test_tfidf)

# Accuracy calculations
train_acc = accuracy_score(y_train, y_train_pred)
val_acc = accuracy_score(y_val, y_val_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# Output results
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Hyperparameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs'],
    'max_iter': [1000]
}

# Grid search setup
grid_search = GridSearchCV(
    LogisticRegression(),
    param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Train grid search on training data
grid_search.fit(X_train_tfidf, y_train)

# Best parameters and model
print("Best Logistic Regression Parameters:", grid_search.best_params_)
best_lr_model = grid_search.best_estimator_

# Predictions using tuned model
y_train_pred_tuned = best_lr_model.predict(X_train_tfidf)
y_val_pred_tuned = best_lr_model.predict(X_val_tfidf)
y_test_pred_tuned = best_lr_model.predict(X_test_tfidf)

# Accuracy calculations for tuned model
train_acc_tuned = accuracy_score(y_train, y_train_pred_tuned)
val_acc_tuned = accuracy_score(y_val, y_val_pred_tuned)
test_acc_tuned = accuracy_score(y_test, y_test_pred_tuned)

# Output tuned model results
print(f"TUNED Logistic Regression Train Accuracy: {train_acc_tuned:.4f}")
print(f"TUNED Logistic Regression Validation Accuracy: {val_acc_tuned:.4f}")
print(f"TUNED Logistic Regression Test Accuracy: {test_acc_tuned:.4f}")

4) Naive Bayes

In [ ]:
# Split into train+validation and test sets first (80% train_val, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Then split train+validation into train and validation sets (80/20 split of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)  # Final: 60% train, 20% val, 20% test

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Naive Bayes Model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Predictions
y_train_pred = nb_model.predict(X_train_tfidf)
y_val_pred = nb_model.predict(X_val_tfidf)
y_test_pred = nb_model.predict(X_test_tfidf)

# Accuracy calculations
train_acc = accuracy_score(y_train, y_train_pred)
val_acc = accuracy_score(y_val, y_val_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# Output results
print(f"Naive Bayes Train Accuracy: {train_acc:.4f}")
print(f"Naive Bayes Validation Accuracy: {val_acc:.4f}")
print(f"Naive Bayes Test Accuracy: {test_acc:.4f}")

# Parameter grid for Naive Bayes
nb_param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1.0, 2.0]  # Laplace smoothing
}

# Set up GridSearchCV
nb_grid_search = GridSearchCV(
    MultinomialNB(),
    nb_param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Fit grid search on training data
nb_grid_search.fit(X_train_tfidf, y_train)

# Best parameters and model
print("Best Naive Bayes Hyperparameters:", nb_grid_search.best_params_)
best_nb_model = nb_grid_search.best_estimator_

# Predictions using tuned model
y_train_pred_tuned = best_nb_model.predict(X_train_tfidf)
y_val_pred_tuned = best_nb_model.predict(X_val_tfidf)
y_test_pred_tuned = best_nb_model.predict(X_test_tfidf)

# Accuracy calculations for tuned model
train_acc_tuned = accuracy_score(y_train, y_train_pred_tuned)
val_acc_tuned = accuracy_score(y_val, y_val_pred_tuned)
test_acc_tuned = accuracy_score(y_test, y_test_pred_tuned)

# Output tuned model results
print(f"TUNED Naive Bayes Train Accuracy: {train_acc_tuned:.4f}")
print(f"TUNED Naive Bayes Validation Accuracy: {val_acc_tuned:.4f}")
print(f"TUNED Naive Bayes Test Accuracy: {test_acc_tuned:.4f}")